# Imports

In [1]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (272 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [2]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [3]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

!pip install -U lightrag[ollama]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [4]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display

import time

# Code

In [ ]:
class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, query: str) -> str:
        return self.generator.call({"input_str": query})

    async def acall(self,query: str) -> str:
        return await self.generator.acall({"input_str": query})

In [ ]:
qa_template = r"""<SYS>
You are a helpful assistant Who is the backend of a prompt based interface which simplifies CRM(Customer Relationship Management) tasks. Your tasks include, creating a marketing campaign for a audience, integrating other services like email to notify, creating reports etc., Take suitable prompts as Input.
</SYS>
User: {{input_str}}
You:"""

In [ ]:
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}

qa = SimpleQA(**model)

In [ ]:
query = r"""
Hi, Create a marketing campaign for the annual Sports fest of my college called Varcchas
"""

In [ ]:

output = qa.call(query)
display(Markdown(f"**Answer:** {output.data}"))


**Answer:** Welcome to Varcchas CRM! I'd be happy to help you create a marketing campaign for our annual Sports Fest.

To get started, could you please provide me with some details about the event? 

Here are a few prompts to consider:

1. **Target Audience**: Who do we want to invite (students, faculty members, alumni)?
2. **Campaign Theme**: What's the central idea or message we want to convey through our marketing efforts?
3. **Event Date and Time**: When does Varcchas take place? 
4. **Key Highlights**: Are there any special events, competitions, or attractions that you'd like to showcase?

Please respond with the necessary details, and I'll start crafting a comprehensive marketing campaign for Varcchas 2023!

# RAG

In [6]:
!pip install -U lightrag[ollama] langchain chromadb sentence-transformers PyPDF2 langchain_community

# Cell 2: Import required libraries
import os
from typing import List, Dict, Any
from pathlib import Path
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
import threading
import subprocess

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00


In [7]:
def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

# Download the model
!ollama pull llama3.1:8b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏ 8.0 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏  17 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   1% ▕▏  52 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   1% ▕▏  69 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   2% ▕▏ 113 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   3% ▕▏ 134 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   3% ▕▏ 147 MB/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   4% ▕▏ 178 MB/4.9 GB  178 MB/s   

In [8]:
class TextLoader:
    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            add_start_index=True,
        )

    def load_pdf(self, file_path: str) -> List[str]:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()

        return self.text_splitter.split_text(text)

In [9]:
class VectorStore:
    def __init__(self, embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
        self.db = None

    def create_db(self, texts: List[str], persist_directory: str):
        self.db = Chroma.from_texts(
            texts=texts,
            embedding=self.embeddings,
            persist_directory=persist_directory
        )
        self.db.persist()

    def load_db(self, persist_directory: str):
        self.db = Chroma(
            persist_directory=persist_directory,
            embedding_function=self.embeddings
        )

    def similarity_search(self, query: str, k: int = 3) -> List[str]:
        if not self.db:
            raise ValueError("Database not initialized. Please create or load a database first.")
        return [doc.page_content for doc in self.db.similarity_search(query, k=k)]

In [11]:
class NeurosurgeryRAG(Component):
    def __init__(
        self,
        model_client: ModelClient,
        model_kwargs: dict,
        vector_store: VectorStore,
        max_context_length: int = 2000
    ):
        super().__init__()
        self.vector_store = vector_store
        self.max_context_length = max_context_length
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=self._get_rag_template(),
        )

    def _get_rag_template(self) -> str:
        return """<SYS>
You are a knowledgeable neurosurgery assistant. Use the provided context to answer questions accurately and professionally. If you're unsure or the context doesn't contain relevant information, say so. Always maintain medical accuracy and precision.
</SYS>

Context:
{{context}}

User: {{input_str}}"""

In [12]:
loader = TextLoader(chunk_size=1000, chunk_overlap=200)

# Load your PDF files
# Replace these paths with the actual paths to your PDF files
pdf_paths = [
    "Brazis.pdf",

]

# Process all PDFs
all_chunks = []
for pdf_path in pdf_paths:
    chunks = loader.load_pdf(pdf_path)
    all_chunks.extend(chunks)
    print(f"Processed {pdf_path}: {len(chunks)} chunks created")

Processed Brazis.pdf: 4939 chunks created


In [13]:
vector_store = VectorStore()

# Create and persist the database
persist_dir = "./neurosurgery_db"  # This directory will store your vector database
vector_store.create_db(all_chunks, persist_dir)

<ipython-input-9-a86a25351830>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-9-a86a25351830>:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  self.db.persist()


In [14]:
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}

# Create the RAG instance
rag = NeurosurgeryRAG(
    model_client=model["model_client"],
    model_kwargs=model["model_kwargs"],
    vector_store=vector_store
)

In [17]:

def query_neurosurgery(question: str):
    # Get relevant context from vector store
    context_chunks = vector_store.similarity_search(question)

    # Format context for the model
    formatted_context = "\n\n".join(context_chunks)

    # Generate response
    response = rag.generator.call({
        "input_str": question,
        "context": formatted_context
    })

    # Display retrieved context chunks
    print("Retrieved Context Chunks:")
    print("-" * 80)
    for i, chunk in enumerate(context_chunks, 1):
        print(f"Chunk {i}:")
        print(chunk)
        print("-" * 80)

    # Display final answer
    print("\nGenerated Answer:")
    print("=" * 80)
    print(response.data)
    print("=" * 80)

    return response.data

# Example usage:
query = "Can you tell me about leisions of basal ganglia?"
response = query_neurosurgery(query)

Retrieved Context Chunks:
--------------------------------------------------------------------------------
Chunk 1:
.	Someone	typing	a	letter	may	beconscious	of	the	movements	of	the	fingers	on	the	keyboard	but	is	not	generallyconscious	of	the	shoulder	abduction	and	extension	that	allow	the	fingers	to	flyover	the	keyboard.The	major	anatomic	connections	of	the	basal	ganglia	[8,52]	are	complex(Fig.	19-2)	and	include	several	“closed	circuits”	of	connections[302,316,326].	In	essence,	the	basal	ganglia	consist	of	an	input	zone,	comprising
--------------------------------------------------------------------------------
Chunk 2:
0.ѵLesions	of	the	Basal	GangliaPathologic	processes	affecting	the	basal	ganglia	are	often	diffuse.	When	discrete,they	usually	also	affect	neighboring	structures,	such	as	the	internal	capsule,	thehypothalamus,	and	the	white	matter	of	the	cerebral	hemispheres.	Therefore,except	for	hemiballismus	often	associated	with	damage	to	the	contralateralsubthalamic	nucleus,	correla